In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values

def training(data,values,clf):
    trained = clf.fit(data,values)
    
    return trained
    
def predict(test,true,clf):
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'norm_svr.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+"_norm_svr.xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

def norm(data):
    scaler.fit(data)
    normalized = scaler.transform(data)
    
    return normalized

In [4]:
kernels = RationalQuadratic(length_scale=1.0, alpha=1.0, 
                            length_scale_bounds=(1e-3, 1e3), 
                            alpha_bounds=(1e-3, 1e3))

# CASE 1

In [5]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar)
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) 

(744, 26)


INITIAL MODEL

In [6]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

In [7]:
print("arousal:\t",gpr_a.kernel)
print("arousal:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
arousal:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [8]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [9]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, gpr_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, gpr_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [10]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0103, length_scale=0.207)
valence:	 RationalQuadratic(alpha=0.0135, length_scale=0.0719)


AROUSAL

In [11]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.012247,0.190538,0.494518,-0.319365,-0.415454,-0.113565,-0.642037,0.382573,-0.454567,-0.017574,...,0.497475,5.373489e-02,-0.076619,-0.050936,-0.004373,-0.002217,0.288890,0.431990,-0.180097,-0.580886
1,0.352941,0.264706,0.411765,-0.558824,-0.441176,-0.411765,-0.588235,0.235294,-0.411765,0.352941,...,0.588235,-2.220446e-16,-0.382353,0.235294,-0.088235,-0.029412,-0.058824,0.382353,-0.500000,-0.823529


R2 Score Arousal:	0.67


VALENCE

In [12]:
# df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.225820,0.247975,0.197328,-0.111698,-0.079214,0.033555,-0.228052,0.319648,-0.207793,0.119562,...,0.239213,0.346062,-0.000784,0.110781,0.215404,0.103555,0.174324,0.217511,-0.084373,-0.334537
1,-0.661538,0.476923,0.353846,-0.661538,0.046154,-0.507692,-0.076923,-0.169231,-0.230769,0.569231,...,0.538462,-0.015385,-0.723077,0.230769,-0.323077,-0.138462,0.507692,0.507692,-0.261538,-0.415385


R2 Score Valence: 0.28


REGRESSION

In [13]:
scores_a = regresion(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = regresion(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [14]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.46199151  0.4891282   0.54630226  0.43903648  0.69737876  0.59063482
  0.54492899  0.5853275   0.65095956  0.5929507 ]
R2 Score Valence:	0.56 (+/- 0.08)


VALENCE

In [15]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.04591989  0.01718468  0.04646442 -0.1743074   0.46422922  0.15758722
  0.20406223  0.27650744  0.2655793   0.02273391]
R2 Score Valence:	0.13 (+/- 0.17)


In [16]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 2

In [17]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 34)


INITIAL MODEL

In [18]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

In [19]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [20]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [21]:
# #Predict Arousal
pred_a,df2_a = predict(test_data_a, test_values_a, gpr_a)
# #Predict Valence
pred_v,df2_v = predict(test_data_v, test_values_v, gpr_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [22]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0101, length_scale=0.225)
valence:	 RationalQuadratic(alpha=0.0132, length_scale=0.0767)


AROUSAL

In [23]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.012247,0.190538,0.494518,-0.319365,-0.415454,-0.113565,-0.642037,0.382573,-0.454567,-0.017574,...,0.497475,5.373489e-02,-0.076619,-0.050936,-0.004373,-0.002217,0.288890,0.431990,-0.180097,-0.580886
1,0.352941,0.264706,0.411765,-0.558824,-0.441176,-0.411765,-0.588235,0.235294,-0.411765,0.352941,...,0.588235,-2.220446e-16,-0.382353,0.235294,-0.088235,-0.029412,-0.058824,0.382353,-0.500000,-0.823529


R2 Score Arousal:	0.67


VALENCE

In [24]:
# df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.225820,0.247975,0.197328,-0.111698,-0.079214,0.033555,-0.228052,0.319648,-0.207793,0.119562,...,0.239213,0.346062,-0.000784,0.110781,0.215404,0.103555,0.174324,0.217511,-0.084373,-0.334537
1,-0.661538,0.476923,0.353846,-0.661538,0.046154,-0.507692,-0.076923,-0.169231,-0.230769,0.569231,...,0.538462,-0.015385,-0.723077,0.230769,-0.323077,-0.138462,0.507692,0.507692,-0.261538,-0.415385


R2 Score Valence:	0.27


REGRESSION

In [25]:
scores_a = regresion(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = regresion(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [26]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.47044567  0.47580088  0.53834128  0.44398701  0.6956205   0.55736272
  0.54161769  0.57368614  0.63256902  0.60410416]
R2 Score Valence:	0.55 (+/- 0.07)


VALENCE

In [27]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.05585104  0.03807502  0.03240924 -0.18012408  0.47300756  0.12680563
  0.18900027  0.27085813  0.24324239  0.01268423]
R2 Score Valence:	0.13 (+/- 0.17)


In [28]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(train_a,datafitur,'arousal/') #save model Arousal
save_model(train_v,datafitur,'valence/') #save model Valence

NameError: name 'train_a' is not defined

# CASE 3

In [ ]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

INITAL MODEL

In [ ]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

In [ ]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

TRAIN

In [ ]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [ ]:
# #Predict Arousal
pred_a,df2_a = predict(test_data_a, test_values_a, svr_case1_a)
# #Predict Valence
pred_v,df2_v = predict(test_data_v, test_values_v, svr_case1_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

SET PARAM

In [ ]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

AROUSAL

In [ ]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

VALENCE

In [ ]:
# df3_v = df3_v.round(2)

display(df3_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

REGRESSION

In [ ]:
scores_a = regresion(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = regresion(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [ ]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [ ]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [ ]:
save_predict(df_a,df_v,datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 4

In [ ]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

INITIAL MODEL

In [ ]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Valence

In [ ]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

TRAIN

In [ ]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [ ]:
# #Predict Arousal
pred_a,df2_a = predict(test_data_a, test_values_a, gpr_a)
# #Predict Valence
pred_v,df2_v = predict(test_data_v, test_values_v, gpr_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [ ]:
trained_a.set_param(kernel=trained_a.kernel_)
trained_v.set_param(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

AROUSAL

In [ ]:
# df4_a = df4_a.round(2)

display(df4_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

VALENCE

In [ ]:
# df4_v = df4_v.round(2)

display(df4_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

REGRESSION

In [ ]:
scores_a = regresion(gpr_a,data,scaled_ar) #Arousal Regression
scores_v = regresion(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [ ]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [ ]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [ ]:
save_predict(df_a,df_v,datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence